In [361]:

import pandas as pd

import plotly
import plotly.graph_objects as go
import plotly.express as px



for sheet_name in ['KEM-PKE','DS']:
    data = pd.read_excel('/Users/kamyar/Downloads/PQC_Round_3_Parameter_Sets.xlsx', sheet_name=sheet_name).sort_values(by=['Candidate', 'Level'])


    subfamilies = data['Subfamily'].unique()

    x_column = 'KEM Public Key Size' if sheet_name == 'KEM-PKE' else 'Public Key Size'
    y_column = 'KEM Ciphertext Size' if sheet_name == 'KEM-PKE' else 'Signature Size'

    subfamily_color_map = {k:v for k,v in zip(subfamilies.tolist(), px.colors.qualitative.Plotly) }

    level_to_symbols_map = {1:1, 2:2, 3:0, 4:3, 5:4}

    level_to_size_map = {i:i*1 + 10 for i in range(1,6)}

    fig = go.Figure()
    # fig = px.scatter(data, 
    #     x=x_column, y=y_column,
    #     log_x=True, log_y=True, 
    #     color="Candidate",
    #     opacity=0.8,
    #     size_max=18,
    #     #facet_col='',
    #     symbol="Level",
    #     size="Level",
    #     width=1200, height=800, 
    #     color_discrete_sequence=px.colors.qualitative.T10, 
    #     hover_name='Candidate',
    #     # title='KEM Ciphertext size vs Public Key size',
    #     labels={
    #         x_column: "Public Key (bytes)",
    #         y_column: "Ciphertext (bytes)",
    #         "Level": "Security Level"
    #     },

    # )

    candidates = data['Candidate'].unique()
    levels = range(1,6)

    candidate_colors = data['Candidate'].map( {v:k for k,v in enumerate(candidates)} )

    for cnd_idx,cnd in enumerate(candidates):
        trace_data = data[(data['Candidate'] == cnd)]
        if not 1 in trace_data['Level'].values:
            d1 = trace_data.iloc[0]
            d1['Level'] = 1
            trace_data.loc[-1] = d1  # adding a row
            trace_data.index = trace_data.index + 1  # shifting index
            trace_data = trace_data.sort_index()  # sorting by index
        fig.add_trace(
            go.Scatter(
                x=trace_data[x_column], y=trace_data[y_column],
                mode = "markers",
                showlegend=True,
                legendgroup=cnd,
                name=f"{cnd} ({trace_data.iloc[0]['Subfamily']})",
                # color_discrete_sequence=px.colors.qualitative.T10, 
                # hover_name='Candidate',
                marker_size = 12, #[ s*2 + 12  for s in  data['Level']],
                marker_symbol = trace_data['Level'].map(level_to_symbols_map),
                marker_color= px.colors.qualitative.T10[cnd_idx],
                text = trace_data['Parameter Set'],
                # marker_size = trace_data['Level'].map(level_to_size_map),
                marker=dict(
                    # size=trace_data['Level'],
                    # sizemin=7,
                    # color=cnd_idx,
                    # sizeref=0.5,
                    opacity=1,
                    line=dict(width=0.5, color='black')
                ),
            )
        )


    for lvl in levels:
        trace_data = data[ (data['Level'] == lvl)]
        fig.add_trace(
            go.Scatter(
                x=trace_data[x_column], y=trace_data[y_column],
                mode = "markers",
                showlegend=True,
                legendgroup=lvl,
                name=f"Security Level {lvl}",
                visible='legendonly',
                marker_colorscale=px.colors.qualitative.T10,
                marker_color='Black',
                marker_size = 12, #[ s*2 + 12  for s in  data['Level']],
                marker_symbol = trace_data['Level'].map({k:v+100 for k,v in level_to_symbols_map.items() }),
                text = trace_data['Parameter Set'],
                # marker_size = trace_data['Level'].map(level_to_size_map),
                marker=dict(
                    # size=trace_data['Level'],
                    # sizemin=7,
                    # color=cnd_idx,
                    # sizeref=0.5,
                    opacity=1,
                    line=dict(width=1, color='orange')
                ),
            )
        )


    fig.update_layout(
        xaxis_type="log", yaxis_type="log",
        width=1200, height=800,
        xaxis = dict(
            tickmode = 'array',
            tickvals =[ 2**(2+i) for i in range(0,24) ],
            tickformat = ',f'
        ),
        yaxis = dict(
            tickmode = 'array',
            tickvals =[ 2**(2+i) for i in range(0,28) ],
            tickformat = ',f'
        ),
    )


    fig.update_xaxes(
        tickangle=45,
        title_text = 'Public Key (bytes)',
        title_font=dict(
            size=22, 
            # family='Courier',
            # color='crimson'
        ),
        tickfont=dict(
            # family='Rockwell', 
            # color='crimson', 
            size=14
        ),

    )

    fig.update_yaxes(
        title_text = 'Ciphertext (bytes)' if sheet_name == 'KEM-PKE' else 'Signature Size (bytes)',
        title_font=dict(
            size=22,
            # family='Courier', 
            # color='crimson'
        ),
        tickfont=dict(
            # family='Rockwell', 
            # color='crimson', 
            size=14
        ),
    )

    # fig.update_layout(
    #     margin=dict(l=0, r=0, t=0, b=0),
    # #     paper_bgcolor="LightSteelBlue",
    # )

    config={
        'displaylogo': False,
        'displayModeBar': True,
        'modeBarButtonsToRemove': ['hoverClosest3d', 'hoverClosestCartesian', 'hoverClosestGeo' , 'hoverClosestGl2d', 'hoverClosestPie' , 
            'lasso2d','toggleSpikelines',' hoverClosestCartesian', 'hoverCompareCartesian'], 
        'responsive': True
    }

    fig.update_layout(
        legend=dict(
            yanchor="top",
            y=0.97,
            xanchor="right",
            x=0.97,
            # itemsizing='constant',
            # traceorder='reversed+grouped',
            borderwidth=4,
            bordercolor='#ffffff',
            # title_font_family="Times New Roman",
            title_font_size=24,
            # title = 'Legend',
            font=dict(
                # family="Courier",
                size=14,
                color="black"
            ),
        )
    )

    plot_name = f'plot_{sheet_name.lower()}'

    fig.show(config=config)
    fig.write_html(f"{plot_name}.html", include_plotlyjs='directory',config=config)
    fig.write_image(f"{plot_name}.svg")
    fig.write_image(f"{plot_name}.pdf")

In [370]:
from plotly.subplots import make_subplots

for sheet_name in ['KEM-PKE','DS']:
    data = pd.read_excel('/Users/kamyar/Downloads/PQC_Round_3_Parameter_Sets.xlsx', sheet_name=sheet_name).sort_values(by=['Level','Candidate', ])


    subfamilies = data['Subfamily'].unique()

    x_column = 'KEM Public Key Size' if sheet_name == 'KEM-PKE' else 'Public Key Size'
    y_column = 'KEM Ciphertext Size' if sheet_name == 'KEM-PKE' else 'Signature Size'



    fig = make_subplots(rows=1, cols=5)
    # fig = go.Figure()
    fig = px.scatter(data, 
        x=x_column, y=y_column,
        log_x=True, log_y=True, 
        color="Candidate",
        opacity=1,
        # size_max=12,
        size=[16 for _ in range(0, len(data))],
        facet_col="Level",
        width=1800, height=800,
        facet_col_spacing=0.015,
        # color_discrete_sequence=px.colors.qualitative.T10, 
        # hover_name='Candidate',
        # title='KEM Ciphertext size vs Public Key size',
        # labels={
        #     x_column: "Public Key (bytes)",
        #     y_column: "Ciphertext (bytes)",
        #     "Level": "Security Level"
        # },

    )

    # candidates = data['Candidate'].unique()
    # levels = range(1,6)

    # candidate_colors = data['Candidate'].map( {v:k for k,v in enumerate(candidates)} )

    # for lvl in levels:
    #     for cnd_idx,cnd in enumerate(candidates):
    #         trace_data = data[(data['Level'] == lvl) & (data['Candidate'] == cnd)]
    #         if len(trace_data) > 0:
    #             fig.add_trace(
    #                 go.Scatter(
    #                     x=trace_data[x_column], y=trace_data[y_column],
    #                     mode = "markers",
    #                     showlegend=True,
    #                     legendgroup=cnd,
    #                     name=f"{cnd} ({trace_data.iloc[0]['Subfamily']})",
    #                     # color_discrete_sequence=px.colors.qualitative.T10, 
    #                     # hover_name='Candidate',
    #                     marker_size = 12, #[ s*2 + 12  for s in  data['Level']],
    #                     marker_symbol = trace_data['Level'].map(level_to_symbols_map),
    #                     marker_color= px.colors.qualitative.T10[cnd_idx],
    #                     text = trace_data['Parameter Set'],
    #                     # marker_size = trace_data['Level'].map(level_to_size_map),
    #                     marker=dict(
    #                         # size=trace_data['Level'],
    #                         # sizemin=7,
    #                         # color=cnd_idx,
    #                         # sizeref=0.5,
    #                         opacity=1,
    #                         line=dict(width=0.5, color='black')
    #                     ),
    #                 ),
    #                 row=1, col=lvl
    #             )


    fig.update_layout(
        xaxis_type="log", yaxis_type="log",
        width=2400, height=800,
        xaxis = dict(
            tickmode = 'array',
            tickvals =[ 2**(3+i) for i in range(0,28) ],
            tickformat = ',f'
        ),
        yaxis = dict(
            tickmode = 'array',
            tickvals =[ 2**(3+i) for i in range(0,18) ],
            tickformat = ',f'
        ),
    )


    fig.update_xaxes(
        tickangle=45,
        title_text = 'Public Key (bytes)',
        title_standoff = 0,
        title_font=dict(
            size=18, 
            # family='Courier',
            # color='crimson'
        ),
        tickfont=dict(
            # family='Rockwell', 
            # color='crimson', 
            size=14
        ),

    )

    fig.update_yaxes(
        title_text = 'Ciphertext (bytes)' if sheet_name == 'KEM-PKE' else 'Signature Size (bytes)',
        title_standoff = 0,
        title_font=dict(
            size=18,
            # family='Courier', 
            # color='crimson'
        ),
        tickfont=dict(
            # family='Rockwell', 
            # color='crimson', 
            size=14
        ),
    )

    # fig.update_layout(
    #     margin=dict(l=0, r=0, t=0, b=0),
    # #     paper_bgcolor="LightSteelBlue",
    # )

    config={
        'displaylogo': False,
        'displayModeBar': True,
        'modeBarButtonsToRemove': ['hoverClosest3d', 'hoverClosestCartesian', 'hoverClosestGeo' , 'hoverClosestGl2d', 'hoverClosestPie' , 
            'lasso2d','toggleSpikelines',' hoverClosestCartesian', 'hoverCompareCartesian'], 
        'responsive': True
    }

    fig.update_layout(
        legend=dict(
            yanchor="top",
            y=0.80,
            xanchor="right",
            x=0.994,
            # itemsizing='constant',
            # traceorder='reversed+grouped',
            borderwidth=1,
            bordercolor='#ffffff',
            # title_font_family="Times New Roman",
            title_font_size=20,
            title = '',
            font=dict(
                # family="Courier",
                size=22,
                color="black"
            ),
        )
    )

    plot_name = f'plot_facets_{sheet_name.lower()}'

    fig.show(config=config)
    fig.write_html(f"{plot_name}.html", include_plotlyjs='directory',config=config)
    fig.write_image(f"{plot_name}.svg")
    fig.write_image(f"{plot_name}.pdf")